In [1]:
# To Dos

# - Make a special case for a library like Vaccine Library: This sheet should get a concept per row in the blue columns, plus a "contains"-like mapping for each code from each vocab
# - Make value set with vaccine codes for Library

# - Make value sets for all vaccine products

# - Deal with duplicate input options? Or maybe just let OCL take the latest one?
#

In [2]:
# Parameters
environment = "staging."
ocl_org = "WHO-Smart-Guidelines"
base_canonical_url = "https://smart.who.int/smart-dak-example-immz/ValueSet/"
main_dak_source = 'smart-dak-example-immz'

# Spreadsheet tab groups, which will be treated differently
info_tabs = ['COVER','READ ME']  # Tabs with information only and should not be imported to OCL (leave empty if none are present)
data_tabs = ['IMMZ.C Client Registration', 'IMMZ.D1 Capture client history', 'IMMZ.D Administer vaccine', 'IMMZ.I Report Generation']  # Tabs with DAK data for processing
library_tab = 'IMMZ.Z Vaccine library'  # Tab containing Libraries of codes (leave blank if none is present)
blue_columns = [
    'Activity ID', 'Data Element ID', 'Data Element Label', 'Description and Definition',
    'Multiple Choice Type \n(if applicable)', 'Data Type', 'Input Options', 'Calculation',
    'Quantity Sub-Type', 'Validation Condition', 'Editable', 'Required',
    'Explain Conditionality', 'Linkages to Aggregate Indicators', 'Notes'
] # These are columns with blue formatting, which are the primary concepts to be loaded. 

In [3]:
# Python Libraries to import
import pandas as pd
import urllib.request
import numpy as np
import json
import openpyxl
import warnings
from collections import defaultdict
import re
from tabulate import tabulate
import unicodedata
import csv



# Suppress openpyxl warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')


In [4]:
# Function for better displaying larger tables
def display_data_dict(data_dict):
    for sheet_name, df in data_dict.items():
        print(f"\n{sheet_name}:")
        print(tabulate(df, headers='keys', tablefmt='pipe', showindex=False))
        print(f"\nShape: {df.shape}")
        print("-" * 80)  # Separator between tables

In [5]:
# Imports spreadsheet data

# # Dataframe with sheet title and url from Google Sheets
# urls_df = pd.DataFrame({
#     'title': ['IMMZ.C Register Client', 'IMMZ.G Pre-vaccination Data', 'IMMZ.G Administer Vaccine', 'IMMZ.J Generate Reports', 'IMMZ.Z Vaccine Library'],
#     'url': [
#         'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=0',
#         'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=536740205',
#         'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=651729522',
#         'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=1146884570',
#         'https://docs.google.com/spreadsheets/d/1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno/export?format=csv&id=1cSPRs69VkkowMD6FQJ33l14dP70SfINP3st3p9eFuno&gid=975478021'
#     ]
# })

# ----------------------------------------------------

# Specify the path to your Excel file
excel_file_path = 'EXAMPLE IMMZ DAK_data dictionary.xlsx'

# Load the Excel workbook
workbook = openpyxl.load_workbook(excel_file_path, read_only=True)

# Get all sheet names
all_sheet_names = workbook.sheetnames

# Create dictionaries to store DataFrames
info_dict = {}
data_dict = {}
library_dict = {}

# Read each sheet into the appropriate dictionary
for sheet_name in all_sheet_names:
    if sheet_name in info_tabs:
        info_dict[sheet_name] = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    elif sheet_name in data_tabs:
        data_dict[sheet_name] = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    elif sheet_name == library_tab:
        library_dict[sheet_name] = pd.read_excel(excel_file_path, sheet_name=sheet_name)
    else:
        print(f"Warning: Sheet '{sheet_name}' not categorized and will be skipped.")

# Close the workbook
workbook.close()

# Print summary
print("\nSummary of processed sheets:")
print(f"Information sheets: {list(info_dict.keys())}")
print(f"Data sheets: {list(data_dict.keys())}")
print(f"Library sheet: {list(library_dict.keys())}")

# Now you can use info_dict, data_dict, and library_dict to access each sheet's data
# For example:
# info_data = info_dict['README']
# client_data = data_dict['IMMZ.C Register Client']
# library_data = library_dict['IMMZ.Z Vaccine Library']
# display(library_dict['IMMZ.Z Vaccine library'])
# display_data_dict(data_dict)



Summary of processed sheets:
Information sheets: ['COVER', 'READ ME']
Data sheets: ['IMMZ.C Client Registration', 'IMMZ.D1 Capture client history', 'IMMZ.D Administer vaccine', 'IMMZ.I Report Generation']
Library sheet: ['IMMZ.Z Vaccine library']


In [6]:
# Standardize column names for standard data dictionary and library tabs - check this output for unexpected column names

def standardize_column_names(data_dict):
    standardized_dict = {}
    for sheet_name, df in data_dict.items():
        # Convert all column names to lowercase
        df.columns = df.columns.str.lower()
        # Replace spaces with underscores
        df.columns = df.columns.str.replace(' ', '_')
        # Remove any special characters (keep only alphanumeric and underscore)
        df.columns = df.columns.str.replace(r'[^\w]', '', regex=True)
        standardized_dict[sheet_name] = df
    return standardized_dict

def standardize_column_name_list(column_names):
    standardized_names = []
    for name in column_names:
        # Convert to lowercase
        name = name.lower()
        # Replace spaces with underscores
        name = name.replace(' ', '_')
        # Remove any special characters (keep only alphanumeric and underscore)
        name = re.sub(r'[^\w]', '', name)
        standardized_names.append(name)
    return standardized_names


def compare_columns(standardized_dicts):
    all_columns = set()
    sheet_columns = {}
    
    # Collect all unique columns and columns for each sheet
    for dict_name, dict_data in standardized_dicts.items():
        for sheet_name, df in dict_data.items():
            full_sheet_name = f"{dict_name} - {sheet_name}"
            sheet_columns[full_sheet_name] = set(df.columns)
            all_columns.update(df.columns)
    
    # Compare columns across sheets
    column_presence = defaultdict(list)
    for column in all_columns:
        for full_sheet_name in sheet_columns.keys():
            if column in sheet_columns[full_sheet_name]:
                column_presence[column].append(full_sheet_name)
    
    # Identify columns not present in all sheets
    all_sheets = set(sheet_columns.keys())
    inconsistent_columns = {col: sheets for col, sheets in column_presence.items() if set(sheets) != all_sheets}
    
    return inconsistent_columns

# Combine data_dict and library_dict
all_dicts = {
    'data_dict': data_dict,
    'library_dict': library_dict
}

# Standardize column names for all dictionaries
standardized_dicts = {dict_name: standardize_column_names(dict_data) for dict_name, dict_data in all_dicts.items()}

# Compare columns across all sheets in all dictionaries
inconsistent_columns = compare_columns(standardized_dicts)

# Print results
print("\nStandardized column names:")
for dict_name, dict_data in standardized_dicts.items():
    print(f"\n{dict_name}:")
    for sheet_name, df in dict_data.items():
        print(f"\n  {sheet_name}:")
        print(f"  {df.columns.tolist()}")

print("\nColumns not present in all sheets:")
for column in sorted(inconsistent_columns.keys()):  # Sort column names alphabetically
    sheets = inconsistent_columns[column]
    print(f"\n{column}:")
    print(f"Present in: {', '.join(sorted(sheets))}")  # Sort sheet names alphabetically
    missing_sheets = set(sheet for dict_data in standardized_dicts.values() for sheet in dict_data.keys()) - set(sheets)
    print(f"Missing from: {', '.join(sorted(missing_sheets))}")  # Sort missing sheet names alphabetically

# Update the original dictionaries with standardized column names
data_dict = standardized_dicts['data_dict']
library_dict = standardized_dicts['library_dict']

# # Data Checks
# print(data_dict) # .to_csv('data_dict.csv', index=False, encoding='utf-8')
# print(library_dict) # .to_csv('library_dict.csv', index=False, encoding='utf-8')


Standardized column names:

data_dict:

  IMMZ.C Client Registration:
  ['activity_id', 'data_element_id', 'data_element_label', 'description_and_definition', 'multiple_choice_type_if_applicable', 'data_type', 'input_options', 'calculation', 'quantity_subtype', 'validation_condition', 'editable', 'required', 'explain_conditionality', 'linkages_to_aggregate_indicators', 'notes', 'icd11code', 'icd11uri', 'icd11comments__considerations', 'icd11_relationship', 'icd10code', 'icd10comments__considerations', 'icd10_relationship', 'loinc_version_268code', 'loinc_version_268comments__considerations', 'loinc_version_268_relationship', 'ichi_beta_3code', 'ichiuri', 'ichicomments__considerations', 'ichi_relationship', 'icfcode', 'icfcomments__considerations', 'icf_relationship', 'snomed_gps_code', 'snomed_gps_comments__considerations', 'snomed_gpsrelationship']

  IMMZ.D1 Capture client history:
  ['activity_id', 'data_element_id', 'data_element_label', 'description_and_definition', 'relevant_ant

In [7]:
# This section lists similar yet different column names, which were likely identified in the last code block. Place the 'correct' names on the right side of each dictionary.

    # Mapping dictionaries for standardizing column names

loinc_relationship_mapping = {
    'loinc_relationship': 'loinc_relationship',
    'loinc_version_268_relationship': 'loinc_relationship',
    'loinc_version_269_relationship': 'loinc_relationship'
}

snomed_gps_comments_mapping = {
    'snomed_gps_code_comments__considerations': 'snomed_gps_comments_considerations',
    'snomed_gps_code_comments_considerations': 'snomed_gps_comments_considerations',
    'snomed_gps_codecomments__considerations': 'snomed_gps_comments_considerations',
    'snomed_gps_comments__considerations': 'snomed_gps_comments_considerations'
}

snomed_gps_relationship_mapping = {
    'snomed_gps_relationship': 'snomed_gps_relationship',
    'snomed_gpsrelationship': 'snomed_gps_relationship'
}

# Combine all mappings into a single dictionary
column_name_mappings = {
    **loinc_relationship_mapping,
    **snomed_gps_comments_mapping,
    **snomed_gps_relationship_mapping
}

# Additional columns that don't need mapping but should be included in the final set
additional_columns = [
    'relevant_antigens' #,
    # 'who_atc_code',
    # 'who_atc_name',
    # 'who_atc_relationship'
]


def consolidate_column_names(columns):
    return [column_name_mappings.get(col, col) for col in columns]
# print(consolidate_column_names(library_dict['IMMZ.Z Vaccine library']))

# Example usage:
# consolidated_df = consolidate_column_names(original_df)

In [8]:
# Parse through Library tab, if available, to produce the concept(s) that will contain the mapped codes
def standardize_column_name_list(column_names):
    standardized_names = []
    for name in column_names:
        name = name.lower()
        name = name.replace(' ', '_')
        name = re.sub(r'[^\w]', '', name)
        standardized_names.append(name)
    return standardized_names

def process_library_tab(library_dict, data_dict):
    if not library_dict:
        print("No Library tab found. Skipping library processing.")
        return data_dict

    library_df = library_dict[list(library_dict.keys())[0]]
    
    # Standardize and consolidate column names in library_df
    library_df.columns = consolidate_column_names(standardize_column_name_list(library_df.columns))
    
    print("Actual columns in Library tab after consolidation:")
    print(library_df.columns.tolist())
    
    # Standardize and consolidate blue columns
    standardized_blue_columns = consolidate_column_names(standardize_column_name_list(blue_columns))
    print("Standardized and consolidated blue columns:")
    print(standardized_blue_columns)

    # Filter the library DataFrame to include only blue columns that are present
    available_blue_columns = [col for col in standardized_blue_columns if col in library_df.columns]
    print("Available blue columns:")
    print(available_blue_columns)

    if not available_blue_columns:
        print("No blue columns found in the Library tab. Skipping processing.")
        return data_dict

    library_df_blue = library_df[available_blue_columns]

    print(f"Rows in Library tab before filtering: {len(library_df)}")
    library_df_blue = library_df_blue.dropna(subset=available_blue_columns, how='all')
    print(f"Rows in Library tab after filtering: {len(library_df_blue)}")

    # Check if all columns in library_df_blue are present in data_dict
    first_data_df = next(iter(data_dict.values()))
    first_data_df.columns = consolidate_column_names(standardize_column_name_list(first_data_df.columns))

    common_columns = [col for col in library_df_blue.columns if col in first_data_df.columns]
    print("Common columns:")
    print(common_columns)

    if not common_columns:
        print("No common columns found between Library tab and data sheets. Skipping processing.")
        return data_dict

    library_df_blue = library_df_blue[common_columns]

    # Change "data_type" from "Codes" to "Coding"
    if 'data_type' in library_df_blue.columns:
        library_df_blue.loc[library_df_blue['data_type'] == 'Codes', 'data_type'] = 'Coding'

    # Add the processed library data to data_dict
    data_dict['Processed Library Data'] = library_df_blue

    print(f"Added {len(library_df_blue)} rows from the Library tab to data_dict.")
    return data_dict

# Process the library tab and update data_dict
data_dict = process_library_tab(library_dict, data_dict)

# Print summary of data_dict after processing
print("\nUpdated summary of data sheets:")
for sheet_name, df in data_dict.items():
    # Standardize and consolidate column names for each DataFrame in data_dict
    df.columns = consolidate_column_names(standardize_column_name_list(df.columns))
    print(f"{sheet_name}: {len(df)} rows")

# Add this line to include additional columns
for df in data_dict.values():
    for col in additional_columns:
        if col not in df.columns and col in library_dict[list(library_dict.keys())[0]].columns:
            df[col] = library_dict[list(library_dict.keys())[0]][col]

# Display the data_dict
# print(data_dict)


Actual columns in Library tab after consolidation:
['activity_id', 'data_element_id', 'data_element_label', 'description_and_definition', 'multiple_choice_type_if_applicable', 'data_type', 'input_options', 'calculation', 'quantity_subtype', 'validation_condition', 'editable', 'required', 'explain_conditionality', 'linkages_to_aggregate_indicators', 'notes', 'icd11code', 'icd11uri', 'icd11comments__considerations', 'icd11_relationship', 'icd10code', 'icd10comments__considerations', 'icd10_relationship', 'loinc_version_268code', 'loinc_version_268comments__considerations', 'loinc_relationship', 'ichi_beta_3code', 'ichiuri', 'ichicomments__considerations', 'ichi_relationship', 'icfcode', 'icfcomments__considerations', 'icf_relationship', 'who_atc_code', 'who_atc_name', 'who_atc_relationship', 'snomed_gps_code', 'snomed_gps_comments_considerations', 'snomed_gps_relationship']
Standardized and consolidated blue columns:
['activity_id', 'data_element_id', 'data_element_label', 'description_a

In [9]:
# Concatenate all DataFrames
df = pd.concat(data_dict.values(), keys=data_dict.keys(), verify_integrity=True)

# Add source column indicating which DataFrame it came from
df['Origin Tab'] = df.index.get_level_values(0)

# Reset index to move source out of index
df = df.reset_index(drop=True)

# Drop unnamed columns
df = df.drop(columns=[col for col in df.columns if 'unnamed' in col.lower()])

# Fix empty values
df = df.fillna('')

# Function to clean and normalize text
def clean_text(text):
    if isinstance(text, str):
        # Replace specific problematic strings
        text = text.replace('Date â‰¤ Current Date', 'Date ≤ Current Date')
        # Replace other known problematic characters
        text = text.replace('â€"', '–')
        # Remove other non-printable characters, but preserve '≤'
        text = re.sub(r'[^\x20-\x7E\n\r\t≤]', '', text)
        # Strip leading and trailing spaces
        text = text.strip()
    return text

# Apply cleaning function to all elements in the DataFrame
df = df.applymap(clean_text)

# Convert all data to strings after cleaning
df = df.astype(str)

# Combine duplicative columns (if they exist)
if 'icd11comments__considerations' in df.columns and 'icd11comments__considerations1' in df.columns:
    df['icd11comments__considerations'] = df.apply(
        lambda row: row['icd11comments__considerations'] if row['icd11comments__considerations'] != '' and row['icd11comments__considerations1'] == ''
        else row['icd11comments__considerations1'] if row['icd11comments__considerations'] == '' and row['icd11comments__considerations1'] != ''
        else row['icd11comments__considerations'] if row['icd11comments__considerations'] == row['icd11comments__considerations1']
        else row['icd11comments__considerations'] + '; ' + row['icd11comments__considerations1'],
        axis=1
    )
    df.drop(columns='icd11comments__considerations1', inplace=True)

# Fix unencoded values
df = df.replace('–', '-', regex=True)

# Export combined DataFrame to a CSV for data checking
df.to_csv('combined.csv', index=False, encoding='utf-8')
#display(df['validation_condition'][9])


In [10]:
# Transforming CSV for OCL import

# Define a dictionary mapping old column names to new names
# col_rename_map = {
#   "Activity ID": "attr:Activity_ID",
#   "Data Element ID": "id",
#   "Data Element Label": "name[1]",
#   "Description and Definition": "description[1]",
#   "Multiple Choice Type (if applicable)": "attr:Multiple_Choice_Type_(if_applicable)",
#   "Data Type": "datatype",
#   "Calculation": "attr:Calculation",
#   "Quantity Sub-Type": "attr:Quantity_Sub-Type",
#   "Validation Condition": "attr:Validation_Condition",
#   "Editable": "attr:Editable",
#   "Required": "attr:Required",
#   "Skip Logic": "attr:Skip_Logic",
#   "Linkages to Aggregate Indicators": "attr:Linkages_to_Aggregate_Indicators",
#   "Notes": "attr:Notes",
#   "ICD-11 URI": "attr:ICD-11_URI",
#   "ICD-11 Comments / Considerations": "attr:ICD-11_Comments_Considerations",
#   "ICD-11 Relationship": "map_type[0]",
#   "ICD-10 Comments / Considerations": "attr:ICD-10_Comments_Considerations",
#   "ICD-10 Relationship": "map_type[1]",
#   "ICD-9 Comments / Considerations": "attr:ICD-9_Comments_Considerations",
#   "ICD-9 Relationship": "map_type[2]",
#   "LOINC version 2.68 Comments / Considerations": "attr:LOINC_version_2.68_Comments_Considerations",
#   "LOINC version 2.68 Relationship": "map_type[3]",
#   "ICHI URI": "attr:ICHI_URI",
#   "ICHI Comments / Considerations": "attr:ICHI_Comments_Considerations",
#   "ICHI Relationship": "map_type[4]",
#   "ICF Comments / Considerations": "attr:ICF_Comments_Considerations",
#   "ICF Relationship": "map_type[5]",
#   "SNOMED GPS Code Comments Considerations": "attr:SNOMED_GPS_Code_Comments_Considerations",
#   "SNOMED GPS Relationship": "map_type[6]",
#   "SNOMED CT International Version Comments / Considerations": "attr:Snomed_CT_International_Version_Comments_Considerations",
#   "SNOMED CT Relationship": "map_type[7]",
#   "HL7 FHIR R4 - Resource": "attr:HL7_FHIR_R4_Resource",
#   "HL7 FHIR R4 - Values": "attr:HL7_FHIR_R4_Values",
#   "HL7 FHIR R4 Relationship": "map_type[8]",
#   "ICD-11 Description": "attr:ICD-11_Description",
#   "WHO ATC Name": "attr:WHO_ATC_Name",
#   "Origin Tab": "attr:Activity_Group",
#   "Input Options": "attr:Input_Options",
#   "ICD-11 Code": "map_to_concept_url[0]",
#   "ICD-10 Code": "map_to_concept_url[1]",
#   "ICD-9 Code": "map_to_concept_url[2]",
#   "LOINC version 2.68 Code": "map_to_concept_url[3]",
#   "ICHI (Beta 3) Code": "map_to_concept_url[4]",
#   "ICF Code": "map_to_concept_url[5]",
#   "SNOMED GPS Code": "map_to_concept_url[6]",
#   "SNOMED CT International Version Code": "map_to_concept_url[7]",
#   "HL7 FHIR R4 Code": "map_to_concept_url[8]",
#   "WHO ATC Code": "map_to_concept_url[9]"
# }

col_rename_map = {
    "activity_id": "attr:Activity_ID",
    "data_element_id": "id",
    "data_element_label": "name[1]",
    "description_and_definition": "description[1]",
    "multiple_choice_type_if_applicable": "attr:Multiple_Choice_Type_(if_applicable)",
    "data_type": "datatype",
    "input_options": "attr:Input_Options",
    "calculation": "attr:Calculation",
    "quantity_subtype": "attr:Quantity_Sub-Type",
    "validation_condition": "attr:Validation_Condition",
    "editable": "attr:Editable",
    "required": "attr:Required",
    "explain_conditionality": "attr:Explain_Conditionality",
    "linkages_to_aggregate_indicators": "attr:Linkages_to_Aggregate_Indicators",
    "notes": "attr:Notes",
    "icd11code": "map_to_concept_url[0]",
    "icd11uri": "attr:ICD-11_URI",
    "icd11comments__considerations": "attr:ICD-11_Comments_Considerations",
    "icd11_relationship": "map_type[0]",
    "icd10code": "map_to_concept_url[1]",
    "icd10comments__considerations": "attr:ICD-10_Comments_Considerations",
    "icd10_relationship": "map_type[1]",
    "loinc_version_268code": "map_to_concept_url[3]",
    "loinc_version_268comments__considerations": "attr:LOINC_version_2.68_Comments_Considerations",
    "loinc_version_268_relationship": "map_type[3]",
    "ichi_beta_3code": "map_to_concept_url[4]",
    "ichiuri": "attr:ICHI_URI",
    "ichicomments__considerations": "attr:ICHI_Comments_Considerations",
    "ichi_relationship": "map_type[4]",
    "icfcode": "map_to_concept_url[5]",
    "icfcomments__considerations": "attr:ICF_Comments_Considerations",
    "icf_relationship": "map_type[5]",
    "snomed_gps_code": "map_to_concept_url[6]",
    "snomed_gps_comments_considerations": "attr:SNOMED_GPS_Code_Comments_Considerations",
    "snomed_gps_relationship": "map_type[6]",
    "relevant_antigens": "attr:Relevant_Antigens",
    "snomed_gps_code_comments_considerations": "attr:SNOMED_GPS_Code_Comments_Considerations",
    "snomed_gps_code_comments__considerations": "attr:SNOMED_GPS_Code_Comments_Considerations",
    "who_atc_code":"map_to_concept_url[9]",
    "who_atc_name":"attr:WHO_ATC_Name",
    "who_atc_relationship":"map_type[9]",
    "Origin Tab": "attr:Activity_Group"
}

# Rename columns using the mapping
df = df.rename(columns=col_rename_map)

# Set 'attr:Input_Options' to empty if same as 'Data Element ID'
df['attr:Input_Options'] = df.apply(lambda row: '' if row['attr:Multiple_Choice_Type_(if_applicable)'] == "Input Option" else row['attr:Input_Options'], axis=1)
#display(df)
#df.to_csv('test.csv', index=False, encoding='utf-8')

In [11]:
# Prepping mappings for OCL CSV

# Mapping prefixes (Column, prefix)
prefix_dict = {
  "map_to_concept_url[0]": "/orgs/WHO/sources/ICD-11-WHO/concepts/",
  "map_to_concept_url[1]": "/orgs/WHO/sources/ICD-10-WHO/concepts/",
  "map_to_concept_url[2]": "/orgs/WHO/sources/ICD-9-WHO/concepts/",
  "map_to_concept_url[3]": "/orgs/Regenstrief/sources/LOINC/concepts/",
  "map_to_concept_url[4]": "/orgs/WHO/sources/WHO-ICHI/concepts/",
  "map_to_concept_url[5]": "/orgs/WHO/sources/WHO-ICF/concepts/",
  "map_to_concept_url[6]": "/orgs/SNOMED-International/sources/SNOMED-GPS/concepts/",
  "map_to_concept_url[7]": "/orgs/IHTSDO/sources/SNOMED-CT/concepts/",
  "map_to_concept_url[9]": "/orgs/WHO/sources/WHOATC/concepts/"
}

# For each column col:
#     It checks if the column exists in the DataFrame using if col in df.columns.
#     If the column exists, it applies a lambda function to the column using df[col].apply(lambda x: ...).
#     The lambda function checks if the string 'Not classifiable' is present in the value x. If it is, it replaces the value with an empty string ''.
#     If 'Not classifiable' is not present, it splits the value x using the space character ' ' and takes the first part str(x).split(' ')[0].

for col, prefix in prefix_dict.items():
    # Check if column exists in the DataFrame
    if col in df.columns:
        df[col] = df[col].apply(lambda x: '' if 'Not classifiable' in str(x) else (prefix + str(x).split(' ')[0] if str(x) else ''))

        # Convert map types to match OCL? Nah maybe later.
        
### Deal with HL7 codes to tie them to a code system

# Dictionary with column value and system URL
url_dict = {
    "male": "/orgs/HL7/sources/administrative-gender/",
    "female": "/orgs/HL7/sources/administrative-gender/",
    "other": "/orgs/HL7/sources/administrative-gender/",
    "unknown": "/orgs/HL7/sources/administrative-gender/",
    "http://hl7.org/fhir/uv/ips/ValueSet/vaccines-gps-uv-ips": "http://hl7.org/fhir/uv/ips/ValueSet/vaccines-gps-uv-ips",
    "http://hl7.org/fhir/uv/ips/ValueSet/whoatc-uv-ips": "http://hl7.org/fhir/uv/ips/ValueSet/whoatc-uv-ips",
    "http://hl7.org/fhir/ValueSet/immunization-route": "http://hl7.org/fhir/ValueSet/immunization-route",
    "complete | pending | error": "https://build.fhir.org/valueset-measure-report-status.html",
    "summary": "https://build.fhir.org/valueset-measure-report-type.html",
    "increase": "/orgs/fhir-hl7-test/sources/measure-improvement-notation/",
    "decrease": "/orgs/fhir-hl7-test/sources/measure-improvement-notation/",
    "numerator": "/orgs/fhir-hl7-test/sources/measure-population/",
    "denominator": "/orgs/fhir-hl7-test/sources/measure-population/"
}

# # Update the column 'map_to_concept_url[8]'
# df['map_to_concept_url[8]'] = df.apply(lambda row: '' if 'Not classifiable' in str(row['map_to_concept_url[8]']) else
#                                       (url_dict.get(row['map_to_concept_url[8]'], '') + 'concepts/' + row['map_to_concept_url[8]']) if '/orgs/' in url_dict.get(row['map_to_concept_url[8]'], '') else
#                                       url_dict.get(row['map_to_concept_url[8]'], ''), axis=1)

# # Fixes blank map types for HL7 FHIR concepts and assigns them as "Related to"
# df.loc[(df['map_to_concept_url[8]'] != '') & (df['map_type[8]'] == ''), 'map_type[8]'] = 'Related to'

# # Create map_type[9] column based on map_to_concept_url[9]
# map_to_col_9 = "map_to_concept_url[9]"
# map_type_col_9 = "map_type[9]"

# # Makes a mapping column for WHO ATC and sets them to "Equivalent" (which applies only to WHO ATC drug codes) 
# df[map_type_col_9] = ""
# df.loc[df[map_to_col_9].notnull(), map_type_col_9] = "Equivalent"
# display(df)



In [12]:
# Misc Cleanup
# Function to check if a column exists
def column_exists(df, column_name):
    return column_name in df.columns

# Function to add "/" to the end of non-blank values
def add_trailing_slash(value):
    if value and not value.endswith("/"):
        return value + "/"
    return value

# Assign Map Type if mapping is present
# Iterate through each column of map_to_concept_url and map_type
for i in range(10):
    map_to_col = f"map_to_concept_url[{i}]"
    map_type_col = f"map_type[{i}]"
   
    if column_exists(df, map_to_col) and column_exists(df, map_type_col):
        # Update map_type based on map_to_concept_url
        df.loc[(df[map_to_col] != '') & (df[map_type_col] == ''), map_type_col] = 'Unspecified map type'
        
        # Add "/" to the end of non-blank values in map_to_concept_url
        df[map_to_col] = df[map_to_col].apply(add_trailing_slash)

# Remove Map Type if no mapping is present  
for i in range(10):
    map_to_col = f"map_to_concept_url[{i}]"
    map_type_col = f"map_type[{i}]"
   
    if column_exists(df, map_to_col) and column_exists(df, map_type_col):
        df.loc[df[map_to_col].apply(lambda x: x.strip() == '') | df[map_to_col].isnull(), map_type_col] = ""

In [13]:
# Create OCL-required columns that don't have special logic
df['resource_type'] = 'Concept'
df['source'] = main_dak_source
df['owner_id'] = ocl_org
df['owner_type'] = 'Organization'
df['retired'] = 'FALSE'
df['name_type[1]'] = 'Fully Specified'

In [14]:
# Relate input options back to their respective question
df_inputs = df

# Create new columns to store the mapping values and types
df_inputs['map_from_concept_url[10]'] = np.nan
df_inputs['map_to_concept_url[10]'] = np.nan
df_inputs['map_type[10]'] = np.nan
df_inputs['concept_class'] = np.nan

# Initialize a variable to keep track of the last seen "Coding" row's id
last_coding_id = None

# Iterate through the dataframe
for index, row in df_inputs.iterrows():
    if row['datatype'] == 'Coding':
        # Update the last seen "Coding" row's id
        last_coding_id = row['id']
    elif row['datatype'] == 'Codes' and pd.notna(last_coding_id):
        # Only process 'Codes' rows when there's a previous 'Coding' row
        df_inputs.at[index, 'map_type[10]'] = "Q-AND-A"
        df_inputs.at[index, 'map_to_concept_url[10]'] = f"/orgs/{row['owner_id']}/sources/{row['source']}/concepts/{row['id']}/"
        df_inputs.at[index, 'map_from_concept_url[10]'] = f"/orgs/{row['owner_id']}/sources/{row['source']}/concepts/{last_coding_id}/"
        # df_inputs.at[index, 'concept_class'] = 'Input Option'

# # Display the first few rows of the modified dataframe to verify the changes
# print(df_inputs[['datatype', 'id', 'map_from_concept_url[10]', 'map_to_concept_url[10]', 'map_type[10]', 'concept_class']].head(20))

# # Optional: Display summary of non-null values in each column
# print("\nCount of non-null values in each column:")
# print(df_inputs[['map_from_concept_url[10]', 'map_to_concept_url[10]', 'map_type[10]', 'concept_class']].notna().sum())

# Display the first few rows of the modified dataframe to verify the changes
# print(df_inputs[['datatype', 'id', 'map_from_concept_url[10]', 'map_type[10]', 'map_to_concept_url[10]']].head(10))

    # Generate numerical IDs for rows without an existing 'id' value, if needed, then create Q-and-A mappings for those rows to connect them back to the data element (this step is dependent on spreadsheet order) 
# df_inputs.loc[df_inputs['id'] == '', 'id'] = range(1, 1 + len(df_inputs[df_inputs['id'] == '']))
# df_inputs['map_to_concept_url[10]'] = "/orgs/" + df_inputs['owner_id'].astype(str) + "/sources/" + df_inputs['source'].astype(str) + "/concepts/" + df_inputs['id'].astype(str) + "/"
# df_inputs['map_from_concept_url[10]'] = "/orgs/" + df_inputs['owner_id'].astype(str) + "/sources/" + df_inputs['source'].astype(str) + "/concepts/" + df_inputs['map_from_concept_url[10]'].replace('', method='ffill') + "/"
# df_inputs.loc[df_inputs['map_from_concept_url[10]'] == "/orgs/" + df_inputs['owner_id'].astype(str) + "/sources/" + df_inputs['source'].astype(str) + "/concepts/" + df_inputs['id'].astype(str) + "/", ['map_type[10]', 'map_from_concept_url[10]', 'map_to_concept_url[10]']] = ''

df_inputs['concept_class'] = df_inputs.apply(lambda x: 'Data Element' if x['map_type[10]'] != 'Q-AND-A' else 'Input Option', axis=1)

# df_inputs.to_csv('test.csv', index=False, encoding='utf-8')


In [15]:
### Make df_v, which is a list of value sets (made of input options) to be loaded into OCL

# Copies dataframe and subsets only to unique values that belong in valuesets
df_v = df_inputs[df_inputs['map_from_concept_url[10]'] != ''].drop_duplicates(subset=['map_from_concept_url[10]']).copy()
df_v = df_v[['map_from_concept_url[10]']].copy()
df_v['id'] = df_v['map_from_concept_url[10]'].str.rsplit('/', n=2).str[-2]
df_v['lookup'] = df_v['map_from_concept_url[10]'].str.rsplit('/', n=2).str[-2]

# Drop rows where 'id' is NaN
df_v = df_v.dropna(subset=['id'])

df_v = df_v.drop_duplicates(subset='id').copy()


# Merge the two dataframes based on the 'id' column
merged_df = pd.merge(df_v, df_inputs, left_on='lookup', right_on='id', how='left')

    # Access the value of the matching 'id' from the 'df_inputs' dataframe
matching_value = merged_df['name[1]'].values[0]  # Retrieves the first matching value

    # Keep only the specified columns and rename them
merged_df = merged_df[['id_x', 'name[1]']].rename(columns={'id_x': 'id', 'name[1]': 'name'})

# Change/update columns for Bulk Import
merged_df['full_name'] = 'Values for: ' + merged_df['name']
merged_df['owner'] = ocl_org
merged_df['owner_type'] = "Organization"
merged_df['default_locale'] = "en"
merged_df['canonical_url'] = base_canonical_url + merged_df['id']
merged_df['collection_type'] = "Value Set"
merged_df['type'] = "Collection"
merged_df['id'] = merged_df['id']+"-values"
merged_df['name'] = merged_df['id']+": "+ merged_df['name']


# Outputs the value sets to a JSON file
with open('ocl-import-immz-value-sets.json', 'w') as outfile:
    for index, row in merged_df.iterrows():
        json.dump(row.to_dict(), outfile, ensure_ascii=False)
        outfile.write('\n')

# Delete value sets if needed
merged_df['__action'] = "DELETE"

with open('delete-ocl-import-immz-value-sets.json', 'w') as outfile:
    for index, row in merged_df.iterrows():
        json.dump(row.to_dict(), outfile, ensure_ascii=False)
        outfile.write('\n')

In [16]:
###### To Do!
### Makes value set list for Immunization Library

# If there are NaN values, replace them with a suitable value
df_inputs['id'] = df_inputs['id'].fillna('') # Replace NaN with an empty string
    
# df.head()
# Filter the DataFrame to only include rows where the 'id' column starts with 'IMMZ.Z1'
#filtered_df = df[df['id'].str.startswith('IMMZ.Z1')]





In [17]:
### Adds ValueSet URL to df

#Example: https://app.staging.openconceptlab.org/#/orgs/WHO-Smart-Guidelines/collections/IMMZ.E0.DE43-values/
base_url = "https://app."+environment+"openconceptlab.org/#/orgs/WHO-Smart-Guidelines/collections/"

# Create a temporary DataFrame with only the 'id' column from merged_df
temp_df = merged_df[['id']].copy()

# Create a new 'matching_ID' column by removing "-values" from the 'id' column
temp_df['matching_ID'] = temp_df['id'].str.replace('-values', '')

# Merge df with temp_df on 'id' and 'matching_ID' to identify matching IDs
merged_ids = df_inputs.merge(temp_df, left_on='id', right_on='matching_ID', how='inner')

# Create 'attr:ValueSet_URL' column based on matching IDs
df_inputs['attr:ValueSet_URL'] = df_inputs['id'].apply(lambda x: x + '-values' if x in merged_ids['matching_ID'].values else '')
# Concatenate base_url to the existing values in 'attr:ValueSet_URL' column
df_inputs['attr:ValueSet_URL'] = df_inputs.apply(lambda row: base_url + row['attr:ValueSet_URL'] if row['attr:ValueSet_URL'] else '', axis=1)

In [18]:
# Output as OCL-formatted CSV
df_inputs.to_csv('ocl-import-dak.csv', index=False, encoding='utf-8')

# Output test OCL file with only 10 rows
#df.head(10).to_csv('SHORT - ocl-import-immz-dak.csv', index=False, encoding='utf-8')

# Output value set list with only 10 rows (not needed now that it's in JSON format)
#merged_df.head(10).to_csv('SHORT - ocl-import-immz-value-sets.csv', index=False, encoding='utf-8')

In [19]:
### Makes df of references to import into the respective collections

# Copies dataframe and subsets only to unique values that belong in valuesets
df_ref = df_inputs[df_inputs['map_from_concept_url[10]'] != ''].drop_duplicates(subset=['map_from_concept_url[10]','map_to_concept_url[10]']).copy()
df_ref = df_ref[['map_from_concept_url[10]','map_to_concept_url[10]']].copy()
df_ref['col_id'] = df_ref['map_from_concept_url[10]'].str.rsplit('/', n=2).str[-2]+"-values"
df_ref['collection_url'] = "/orgs/" + ocl_org +"/collections/" + df_ref['col_id'] + "/"

# Drop rows where "collection_url" is NaN
df_ref = df_ref.dropna(subset=['collection_url'])

df_ref = df_ref[['collection_url', 'map_to_concept_url[10]']].rename(columns={'map_to_concept_url[10]': 'data:expressions'})
df_ref["resource_type"] = "Reference"

#df_ref.head(10).to_csv('SHORT - ocl-import-immz-value-set-references.csv', index=False, encoding='utf-8')
#display(df_ref)
df_ref["type"] = "Reference"
output_json = df_ref.apply(lambda x: json.dumps({"type": x['type'], "collection_url": x['collection_url'], "data": {"expressions": [x['data:expressions']]}}), axis=1)

# Print or save the JSON output
# for item in output_json:
#     print(item)

#print(output_json)

# Write the output to a JSON file
with open("ocl-import-immz-value-set-references.json", "w") as outfile:
    for line in output_json:
        outfile.write(line + '\n')
# Short JSON output
# with open("SHORT - ocl-import-immz-value-set-references.json", "w") as outfile:
#     for line in output_json[:9]:
#         outfile.write(line + '\n')


In [20]:
### Make df_vs_vers, which contains lines to create collection versions for the various value sets
# Create a new DataFrame "df_vs_vers" with selected columns from "merged_df" and renamed column
df_vs_vers = merged_df[['owner', 'id']].copy()
df_vs_vers.rename(columns={'id': 'collection','owner': 'owner_id'}, inplace=True)

# Add new columns "resource_type", "id", and "description" with specified values
df_vs_vers['resource_type'] = 'Collection Version'
df_vs_vers['id'] = '0.1.0'
df_vs_vers['description'] = 'Initial Load'

# Display the updated DataFrame "df_vs_vers"
#print(df_vs_vers)

#Output as CSV
#df_vs_vers.to_json('ocl-import-immz-value-set-versions.json', orient='records', lines=True)
df_vs_vers.to_csv('ocl-import-immz-value-set-versions.csv', index=False, encoding='utf-8')